In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

import keras
import keras.backend
from keras import layers
from keras import models
from keras import optimizers
import keras.utils

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sklearn
from sklearn.utils import shuffle

import os

/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages

1.13.1


## Useful Functions

In [2]:
def build_CNN():
    input2 = [layers.Input(shape = (len(X_train[i][0]),)) for i in range(1,len(X_train))]
    input1 = layers.Input(shape = (grid, grid,1))
    x = layers.Conv2D(32, (5, 5), activation = 'relu', padding = 'same')(input1)
    x = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(x)
    x = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(x)
    x = layers.MaxPool2D((2, 2))(x)
    x1 = layers.Flatten()(x)
    if input2 == []:
        x = layers.Dense(64, activation='relu')(x1)
    else:
        x = layers.concatenate(inputs = [x1] + input2, axis = -1)
        x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs= [input1] + input2,
                         outputs = output)
    opt = keras.optimizers.Adam(lr = 0.0005,
                                beta_1 = 0.9,
                                beta_2 = 0.9,
                                amsgrad = False)
    model.compile(loss = 'binary_crossentropy',
                optimizer = opt,
                metrics = ['binary_crossentropy', 'accuracy'])
    return model

In [4]:
def build_DNN():
    input2 = [layers.Input(shape = (len(X_train[i][0]),)) for i in range(0,len(X_train))]
    if len(input2) == 1:
        x = layers.Dense(64,activation='relu')(input2[0])
    else:
        x = layers.concatenate(inputs = input2, axis = -1)
        x = layers.Dense(64, activation = 'relu')(x)
    output = layers.Dense(2, activation = 'softmax')(x)
    model = models.Model(inputs = input2, 
                         outputs = output)
    opt=keras.optimizers.Adam(lr = 0.0005,
                              beta_1 = 0.9,
                              beta_2 = 0.9,
                              amsgrad = False)
    model.compile(loss = 'binary_crossentropy',
                optimizer = opt,
                metrics = ['binary_crossentropy', 'accuracy'])
    return model

In [5]:
def build_XY(features,label,dic):
    X = [dic[key] for key in features]
    Y = [dic[key] for key in label]
    dim = [ele.shape+(1,) for ele in X]
    for i in range(0,len(features)):
        X[i] = X[i].reshape(dim[i])
    return X,Y

## Import data:
Using Train and Test set produced by preprocessing notebook.

In [6]:
grid = 16
data_train = np.load('../data/ShowJet_full_train.npz')

In [7]:
n_train = len(data_train['jetPt'])
print(n_train)

1696322


In [8]:
feat_all = [key for key in data_train.keys()];feat_all.remove('labels');feat_all

['jetImages',
 'jetPt',
 'jetEta',
 'jetPhi',
 'tau21',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'jetpull']

# Build CNN

## Build Model with only Images

In [9]:
# features will be loaded into X
features = ['jetImages']

# label into Y
label = ['labels']

In [11]:
X_train, Y_train = build_XY(features,label,data_train)

In [13]:
CNN = build_CNN()

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 9ms/step
[[0.50002486 0.4999752 ]
 [0.50006497 0.49993497]
 [0.5000201  0.49997988]
 [0.5000378  0.49996218]
 [0.5000567  0.49994338]
 [0.5000075  0.49999255]
 [0.5000034  0.4999966 ]
 [0.50011414 0.4998858 ]
 [0.49999133 0.50000864]
 [0.5001163  0.4998837 ]]
[0.6932005882263184, 0.6932005882263184, 0.5]


In [12]:
checkpoint_path = "model/CNN_sub_0.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 3
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS, 
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 1357057 samples, validate on 339265 samples
Epoch 1/3
1357057/1357057 [==============================] - 950s 700us/step - loss: 0.5700 - binary_crossentropy: 0.5700 - acc: 0.7139 - val_loss: 0.8721 - val_binary_crossentropy: 0.8721 - val_acc: 0.4338

Epoch 00001: val_loss improved from inf to 0.87208, saving model to model/CNN_sub_0.h1
Epoch 2/3
1357057/1357057 [==============================] - 941s 693us/step - loss: 0.5450 - binary_crossentropy: 0.5450 - acc: 0.7355 - val_loss: 0.7963 - val_binary_crossentropy: 0.7963 - val_acc: 0.4724

Epoch 00002: val_loss improved from 0.87208 to 0.79631, saving model to model/CNN_sub_0.h1
Epoch 3/3
1357057/1357057 [==============================] - 942s 694us/step - loss: 0.5412 - binary_crossentropy: 0.5412 - acc: 0.7405 - val_loss: 0.8012 - val_binary_crossentropy: 0.8012 - val_acc: 0.5281

Epoch 00003: val_loss did not improve from 0.79631


## Build Model with all Features

### Select Features for Model

Show all possible features

In [15]:
# features will be loaded into X
features = feat_all

# label into Y
label = ['labels']

In [16]:
X_train, Y_train = build_XY(features,label,data_train)

Initialize model:

In [17]:
CNN = build_CNN()

Test model's prediction $before$ training:

In [18]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 7ms/step
[[0.48357704 0.5164229 ]
 [0.48567775 0.5143223 ]
 [0.48854178 0.5114582 ]
 [0.48936164 0.5106383 ]
 [0.4840711  0.5159289 ]
 [0.49210805 0.50789195]
 [0.4878     0.51219994]
 [0.48915666 0.51084334]
 [0.48394787 0.5160522 ]
 [0.48813173 0.5118683 ]]
[0.697828471660614, 0.697828471660614, 0.4000000059604645]


Now train! (warning: if building CNN, computer tends to get loud)

In [17]:
checkpoint_path = "model/CNN_all.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 4
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Train on 1357057 samples, validate on 339265 samples
Epoch 1/4
1357057/1357057 [==============================] - 974s 718us/step - loss: 0.4427 - binary_crossentropy: 0.4427 - acc: 0.7981 - val_loss: 0.6956 - val_binary_crossentropy: 0.6956 - val_acc: 0.6415

Epoch 00001: val_loss improved from inf to 0.69556, saving model to model/CNN_all.h1
Epoch 2/4
1357057/1357057 [==============================] - 987s 728us/step - loss: 0.4231 - binary_crossentropy: 0.4231 - acc: 0.8104 - val_loss: 0.5873 - val_binary_crossentropy: 0.5873 - val_acc: 0.7019

Epoch 00002: val_loss improved from 0.69556 to 0.58728, saving model to model/CNN_all.h1
Epoch 3/4
1357057/1357057 [==============================] - 1043s 769us/step - loss: 0.4184 - binary_crossentropy: 0.4184 - acc: 0.8126 - val_loss: 0.6007 - val_binary_crossentropy: 0.6007 - val_acc: 0.6988

Epoch 00003: val_loss did not improve from 0.58728
Epoch 4/4
1357057/1357057 [==============================] - 1283s 945us/step - loss: 0.4165 - bi

# Build 10 Models with one expert Variable each

In [18]:
for ii in range(1,len(feat_all)):
    # Select features and load into X and Y
    features = ['jetImages'] + [feat_all[ii]]
    label = ['labels']
    X_train,Y_train = build_XY(features,label,data_train)
    
    # Build Model
    circleCNN = build_CNN()
    
    # Fit 
    checkpoint_path = "model/CNN_sub_"+str(ii)+".h1"
    if not os.path.exists("model"):
        os.mkdir("model")
    model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                       verbose = 1, save_best_only = True, 
                                                       save_weights_only = False, mode = 'auto', 
                                                       period = 1)    
    EPOCHS = 3
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)
    history = circleCNN.fit(
        X_train, Y_train,
        epochs = EPOCHS,
        validation_split = 0.2,
        verbose = 1,
        callbacks = [early_stop, model_checkpoint])
    circleCNN.save(checkpoint_path)

Train on 1357057 samples, validate on 339265 samples
Epoch 1/3
1357057/1357057 [==============================] - 1088s 801us/step - loss: 0.5608 - binary_crossentropy: 0.5608 - acc: 0.7201 - val_loss: 0.8409 - val_binary_crossentropy: 0.8409 - val_acc: 0.4888

Epoch 00001: val_loss improved from inf to 0.84089, saving model to model/CNN_sub_1.h1
Epoch 2/3
1357057/1357057 [==============================] - 1237s 912us/step - loss: 0.5344 - binary_crossentropy: 0.5344 - acc: 0.7416 - val_loss: 0.7971 - val_binary_crossentropy: 0.7971 - val_acc: 0.5348

Epoch 00002: val_loss improved from 0.84089 to 0.79709, saving model to model/CNN_sub_1.h1
Epoch 3/3
1357057/1357057 [==============================] - 1223s 902us/step - loss: 0.5295 - binary_crossentropy: 0.5295 - acc: 0.7461 - val_loss: 0.7386 - val_binary_crossentropy: 0.7386 - val_acc: 0.5640

Epoch 00003: val_loss improved from 0.79709 to 0.73860, saving model to model/CNN_sub_1.h1
Train on 1357057 samples, validate on 339265 sample


Epoch 00003: val_loss improved from 0.82751 to 0.82061, saving model to model/CNN_sub_9.h1
Train on 1357057 samples, validate on 339265 samples
Epoch 1/3
1357057/1357057 [==============================] - 967s 713us/step - loss: 0.5739 - binary_crossentropy: 0.5739 - acc: 0.7103 - val_loss: 0.7832 - val_binary_crossentropy: 0.7832 - val_acc: 0.4769

Epoch 00001: val_loss improved from inf to 0.78316, saving model to model/CNN_sub_10.h1
Epoch 2/3
1357057/1357057 [==============================] - 964s 711us/step - loss: 0.5492 - binary_crossentropy: 0.5492 - acc: 0.7318 - val_loss: 0.7811 - val_binary_crossentropy: 0.7811 - val_acc: 0.5347

Epoch 00002: val_loss improved from 0.78316 to 0.78114, saving model to model/CNN_sub_10.h1
Epoch 3/3
1357057/1357057 [==============================] - 966s 712us/step - loss: 0.5429 - binary_crossentropy: 0.5429 - acc: 0.7386 - val_loss: 0.8319 - val_binary_crossentropy: 0.8319 - val_acc: 0.4943

Epoch 00003: val_loss did not improve from 0.78114


# Train DNN
where we only use expert variables

## Model with all Expert Variables

In [102]:
features = feat_all[1:]
label = ['labels']
X_train,Y_train = build_XY(features,label,data_train)
# Build Model
circleCNN = build_DNN()
# Fit 
checkpoint_path = "model/DNN_all.h1"
if not os.path.exists("model"):
    os.mkdir("model")
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                               verbose = 1, save_best_only = True, 
                               save_weights_only = False, mode = 'auto', 
                               period = 1)    
EPOCHS = 4
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)
history = circleCNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])
circleCNN.save(checkpoint_path)

Train on 714729 samples, validate on 178683 samples
Epoch 1/4
714729/714729 [==============================] - 29s 41us/step - loss: 0.4268 - binary_crossentropy: 0.4268 - acc: 0.8137 - val_loss: 0.5848 - val_binary_crossentropy: 0.5848 - val_acc: 0.7314

Epoch 00001: val_loss improved from inf to 0.58480, saving model to model/DNN_all.h1
Epoch 2/4
714729/714729 [==============================] - 29s 41us/step - loss: 0.4085 - binary_crossentropy: 0.4085 - acc: 0.8254 - val_loss: 0.6051 - val_binary_crossentropy: 0.6051 - val_acc: 0.7192

Epoch 00002: val_loss did not improve from 0.58480
Epoch 3/4
714729/714729 [==============================] - 30s 42us/step - loss: 0.4052 - binary_crossentropy: 0.4052 - acc: 0.8269 - val_loss: 0.5776 - val_binary_crossentropy: 0.5776 - val_acc: 0.7360

Epoch 00003: val_loss improved from 0.58480 to 0.57756, saving model to model/DNN_all.h1
Epoch 4/4
714729/714729 [==============================] - 27s 38us/step - loss: 0.4034 - binary_crossentropy: 